<a href="https://colab.research.google.com/github/skauntey/ALMS-II-sentiment-analysis/blob/main/Copy_ALMS_Model_2_250421.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os, shutil
import kaggle
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline

## 2. Downloading kaggle competition files


In [ ]:
dataset = "sentiment-analysis-on-movie-reviews"
def kaggle_dataset_download(dataset):
    kaggle.api.authenticate()
    kaggle.api.competition_download_files(dataset, path= str(os.getcwd())+"/dataset/", force = True)
    print ('Data downloaded!')

kaggle_dataset_download(dataset)

In [ ]:
## In order to download the dataset make sure that kaggle.json file is available in ~/.kaggle/kaggle.json
# 'Kaggle.json' Downloading instructions are available under 'API Credential' heading of the github page
# https://github.com/Kaggle/kaggle-api

import zipfile
#Define file to download

dataset = "sentiment-analysis-on-movie-reviews"

def kaggle_dataset_download(dataset):
    kaggle.api.authenticate()
    kaggle.api.competition_download_files(dataset, path= str(os.getcwd())+"/dataset/", force = True)
    print ('Data downloaded!')

def unzip_kaggle_file():

    # file_path
    zipped_file = os.listdir('dataset')[0]
    file_path = os.path.join(str(os.getcwd()), 'dataset', zipped_file)
    folder_path = os.path.join(str(os.getcwd()), 'dataset')

    # making sure that previously downloaded file, if any, is deleted before
    for file in os.listdir(folder_path):
        path = os.path.join(os.getcwd(), 'dataset', file)
        if path.split('.')[-1] != "zip":
            try:
                shutil.rmtree(path)
            except:
                raise

    # Unzipping the file
    with zipfile.ZipFile(file_path) as zip_file:
        for member in zip_file.namelist():
            if member.split('.')[-1] == "zip":
                fdir = member.split('.')[0]
                zip_file.extract(member, path= os.path.join('dataset/'+fdir))
                # extracting individual train and test files in their respective folders
                zippedfile_name = os.listdir(os.path.join(str(os.getcwd()+'/dataset/'+fdir+'/')))
                zippedfile = os.path.join(str(os.getcwd())+'/dataset/'+ fdir, zippedfile_name[0])
                with zipfile.ZipFile(zippedfile, mode='r') as tsv_zip:
                    tsv_zip.extractall(path = os.path.join('dataset/'+fdir+'/'))

                os.remove(zippedfile)
            else:
                continue

    os.remove(os.path.join(str(os.getcwd()), 'dataset', zipped_file))
    print('Kaggle file is downloaded and unzipped!')


kaggle_dataset_download(dataset)
unzip_kaggle_file()

In [ ]:
"""mport wget

url = 'https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/10025/32092/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1619688923&Signature=DOK4fkmvfo1EiqTTeKgw1e%2BOoezuCc6HxbI4zXT%2FKGIhsvmxFtwU22aUAq5NvA9WfkKKtXKP%2BGXl506giWgA9J3zgGpeAiRyPWK5Ik3o%2FGCZm%2B5Gw0oQUQN39sbdSD4a5jqmsKB2mDNPF3Q5DbVFsupRY1mUGCVooXC95TD1BczWgAYBCT%2BS4WfVcJanVB7MpPuohkZUDy6rI1uYe0CH%2Fggfe%2FsHXjjUtLIvV0TBJ0hq7g506jI30Gyz7YkOLgD5z1NtS0UrHxMPYq8z31r7sFaHvpvK%2FKuPpXROQ8%2F%2B%2FxZ%2BTxEFIQuQNRbymzsEQjV8XMLKpJt4NnhpnFAWjHge%2Bg%3D%3D&response-content-disposition=attachment%3B+filename%3Dsentiment-analysis-on-movie-reviews.zip'

def dataset_download_and_unzip(url):
    import os
    import wget
    import zipfile

    # creating directory and downloading data
    if not os.path.isdir('dataset'):
        os.mkdir('dataset/')
    else:
        pass

    # checking the format of the data and removing previously saved files to avoid duplication
    wget.download(url, os.path.join(str(os.getcwd()), 'dataset/'))
    ziped_file = r'dataset/sentiment-analysis-on-movie-reviews.zip'
    folder_path = os.path.join(str(os.getcwd()), 'dataset/')
    for files in os.listdir(folder_path):
        path = os.path.join(os.getcwd()+'/'+'dataset/',files)
        if path.split('.')[-1] != "zip":
            try:
                shutil.rmtree(path)
            except:
                raise

    #Unzipping the files
    with zipfile.ZipFile(ziped_file) as zip_file:
        for member in zip_file.namelist():
            if member.split('.')[-1] == "zip":
                fdir = member.split('.')[0]
                zip_file.extract(member, path= os.path.join('dataset/'+fdir))
                # extracting individual train and test files in their respective folders
                zippedfile_name = os.listdir(os.path.join(str(os.getcwd()+'/dataset/'+fdir+'/')))
                zippedfile = os.path.join(str(os.getcwd())+'/dataset/'+ fdir, zippedfile_name[0])
                with zipfile.ZipFile(zippedfile, mode='r') as tsv_zip:
                    tsv_zip.extractall(path = os.path.join('dataset/'+fdir+'/'))

                os.remove(zippedfile)
            else:
                continue

    os.remove(ziped_file)
    print ('Data successfully downloaded and extracted!')

dataset_download_and_unzip(url)

"""

## 3. Converting files to DataFrame

In [ ]:
#C:\Users\eq\Documents\UCL\ELEC0135 ALMS II\ALMS-II-sentiment-analysis\dataset\train
file_path = os.path.join(str(os.getcwd()), 'dataset','train')
file = os.listdir(file_path)[0]
path = os.path.join(file_path, file)
# read the data
train_data = pd.read_csv(path, sep="\t")

original_dataset = pd.DataFrame({ 
    'Text_Reviews': train_data['Phrase'],
    'Labels': train_data['Sentiment']})

pdf_data = original_dataset['Text_Reviews']
pdf_labels = original_dataset['Labels']

print(pdf_data.shape)
print(pdf_labels.shape)
original_dataset.head(10)

#### 3.1 Viewing data

In [ ]:
Label_Class = [('negative', 0), ('somewhat negative', 1), ('neutral', 2), ('somewhat positive', 3), ('positive', 4)]
Label_Class[0][0]

original_dataset['Class'] = [Label_Class[i][0] for i in original_dataset['Labels']]
original_dataset.head()

In [ ]:
original_dataset.info()

In [ ]:
col = ['Text_Reviews','Labels','Class']
original_dataset[col]

In [ ]:
from io import StringIO
Labels_df = original_dataset[['Labels', 'Class']].drop_duplicates().sort_values('Labels')
Labels_id = dict(Labels_df.values)
id_Class = dict(Labels_df[['Labels', 'Class']].values)
Labels_id

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(8,6))
original_dataset.groupby('Class').Text_Reviews.count().plot.bar(ylim=0)
plt.show()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')

features = tfidf.fit_transform(original_dataset.Text_Reviews).toarray()
labels = original_dataset.Labels
features.shape